In [1]:
import csv
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from consul.agent_types.rl_agent import RLAgent
from consul.agent_types.cbr_agent import CaseBased_RLAgent
from war_reason import *
from war_reason_implementations import *

# Load the data

In [2]:
data = pd.read_pickle("FullData.pickle")

In [3]:
# The Game-Theoretic Equilibrium
equilibrium_cols = ['eqTsq', 'eqTnego', 'eqTacqa', 'eqTacqb', 'eqTcapa', 'eqTcapb', 
                'eqTwara', 'eqTwarb']

data["Equilibrium"] = ""
for col in equilibrium_cols:
    data.loc[data[col] == 1, "Equilibrium"] = col

# Cutting out missing data
data = data[data.Equilibrium!=""]

In [4]:
# Observed outcomes
data["Outcome"] = ""

data.loc[((data.cwhost1<2) & (data.cwhost2<2)), "Outcome"] = "StatusQuo"
data.loc[((data.cwhost1>3) & (data.cwhost2>3)), "Outcome"] = "War"
data.loc[((data.cwhost1==data.cwhost2) & (data.cwhost1>1) & 
          (data.cwhost1<4)), "Outcome"] = "Negotiation"
data.loc[((data.cwhost1>data.cwhost2) & (data.cwhost1>1) & 
          (data.cwhost1<4)), "Outcome"] = "Acquiesce_B"
data.loc[((data.cwhost1<data.cwhost2) & (data.cwhost2>1) & 
          (data.cwhost2<4)), "Outcome"] = "Acquiesce_A"
data.loc[((data.cwhost1>data.cwhost2) & (data.cwhost1>3)), "Outcome"] = "Capitulate_B"
data.loc[((data.cwhost1<data.cwhost2) & (data.cwhost2>3)), "Outcome"] = "Capitulate_A"

In [5]:
# OPTIONAL
## Boost all utilities by a fixed factor
for col in data.columns:
    if col[:4] == "wrTu":
        data[col] *= 8

In [6]:
class CollectiveCBRAgent(CBRAgent):
    tree = PointDB(3)
    experience_library = defaultdict(lambda: defaultdict(float))
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tree = self.__class__.tree
        self.experience_library = self.__class__.experience_library

    def set_new_case(self, coords):
        '''
        Find the current working case and set it as the current experience.
        '''
        self.current_coords = coords
        if len(self.experience_library)>2:
            self.current_case = self._get_nearest_neighbor(coords)
        else:
            self.current_case = coords
        self.experience = self.experience_library[self.current_case]

## Fit the CB Model

In [7]:
eugene_world = EUGeneWorld(CollectiveCBRAgent, data, 
                           agent_args={"learning_rate": 0.2, "discount_factor": 0.9},
                           copy_data=True, shuffle_rows=True)

In [8]:
eugene_world.run(report_every=50000, 
                 model_assessors={"Model_Outcome": lambda m: m.current_node,
                                  "Move_Quality": assess_moves})

0
50000
100000
150000
200000
250000
300000
350000
400000
450000


KeyboardInterrupt: 

In [ ]:
import datetime as dt
print dt.datetime.now()

In [ ]:
data = eugene_world.eugene_data

In [ ]:
outcome_matching = {
    "eqTacqa": ["Acquiesce_A"],
    "eqTacqb": ["Acquiesce_B"],
    "eqTnego": ["Negotiate_1", "Negotiate_2"],
    "eqTsq": ["StatusQuo"],
    "eqTwara": ["War_A1", "War_A2"],
    "eqTwarb": ["War_B1", "War_B2"]}

data["Outcome_Check"] = 0
for key, vals in outcome_matching.items():
    data.loc[((data.Equilibrium==key) & 
                   (data.Model_Outcome.isin(vals))), "Outcome_Check"] = 1

outcome_check = data.Outcome_Check
#plt.plot(pd.rolling_mean(outcome_check, 1000))

In [ ]:
mean_outcome = data.groupby("year").aggregate("mean")["Outcome_Check"]
mean_outcome.plot()

In [ ]:
data.Model_Outcome.value_counts()

In [ ]:
data.head()

# Run several iterations of a model

In [ ]:
fitted_data = []
save_cols = ["year", "ccode1", "ccode2", "Equilibrium", "Outcome", "Model_Outcome"]

for i in range(2):
    eugene_world = EUGeneWorld(RLAgent, data, 
                           agent_args={"learning_rate": 0.1, "discount_factor": 0.9},
                           copy_data=True, shuffle_rows=True)
    eugene_world.run(report_every=2000000,
        model_assessors={"Model_Outcome": lambda m: m.current_node})
    next_data = eugene_world.eugene_data.loc[:,save_cols]
    next_data["Iteration"] = i
    fitted_data.append(next_data)
    print(i)

In [ ]:
data = pd.concat(fitted_data)

In [ ]:
data.Iteration.unique()

In [ ]:
outcome_matching = {
    "eqTacqa": ["Acquiesce_A"],
    "eqTacqb": ["Acquiesce_B"],
    "eqTnego": ["Negotiate_1", "Negotiate_2"],
    "eqTsq": ["StatusQuo"],
    "eqTwara": ["War_A1", "War_A2"],
    "eqTwarb": ["War_B1", "War_B2"]}

data["Outcome_Check"] = 0
for key, vals in outcome_matching.items():
    data.loc[((data.Equilibrium==key) & 
                   (data.Model_Outcome.isin(vals))), "Outcome_Check"] = 1

In [ ]:
for group in data.groupby("Iteration"):
    mean_outcome = group[1].groupby("year").aggregate("mean")["Outcome_Check"]
    mean_outcome.plot()

## Does the model prediction predict the actual outcome?

In [ ]:
data = data[(data.cwhost1!=-9) & (data.cwhost2!=-9)]

In [ ]:
model_dummies = pd.get_dummies(data.Model_Outcome)

In [ ]:
model_dummies.columns

In [ ]:
model_dummies["Negotiate"] = (model_dummies.Negotiate_1 + 
                                    model_dummies.Negotiate_2)

model_dummies["War"] = (model_dummies.War_A1 + model_dummies.War_A2 + 
                        model_dummies.War_B1 + model_dummies.War_B2)

In [ ]:
model_dummies["Const."] = 1

In [ ]:
data.Outcome.value_counts()

In [ ]:
data["Acquiesce_A"] = 0
data.loc[data.Outcome=="Acquiesce_A", "Acquiesce_A"] = 1
data["Negotiation"] = 0
data.loc[data.Outcome=="Negotiation", "Negotiation"] = 1
data["War"] = 0
data.loc[data.Outcome=="War", "War"] = 1


In [ ]:
import statsmodels.api as sm

In [ ]:
x_cols = ["Acquiesce_A", "Acquiesce_B", "Negotiate", "War", "Const."]
logit = sm.Logit(data.Acquiesce_A, model_dummies[x_cols])
#logit = sm.Logit(data.Negotiation, model_dummies[x_cols])
result = logit.fit()
print(result.summary())

In [ ]:
x_cols = ["Acquiesce_A", "Acquiesce_B", "Negotiate", "War", "Const."]
logit = sm.Logit(data.Negotiation, model_dummies[x_cols])
#logit = sm.Logit(data.Negotiation, model_dummies[x_cols])
result = logit.fit()
print(result.summary())

In [ ]:
x_cols = ["Acquiesce_A", "Acquiesce_B", "Negotiate", "War", "Const."]
logit = sm.Logit(data.War, model_dummies[x_cols])
result = logit.fit()
print(result.summary())

### Only a subset of the data

In [ ]:
START = 1980
subset = data[data.year>START]
dummies_subset = model_dummies[data.year>START]

In [ ]:
x_cols = ["Acquiesce_A", "Acquiesce_B", "Negotiate", "War", "Const."]
logit = sm.Logit(subset.Acquiesce_A, dummies_subset[x_cols])
result = logit.fit()
print(result.summary())

In [ ]:
x_cols = ["Acquiesce_A", "Acquiesce_B", "Negotiate", "War", "Const."]
logit = sm.Logit(subset.War, dummies_subset[x_cols])
result = logit.fit()
print(result.summary())